In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(6, dtype=tf.int64)
node2 = tf.constant(7, dtype=tf.int64)

print(node1, node2)

Tensor("Const:0", shape=(), dtype=int64) Tensor("Const_1:0", shape=(), dtype=int64)


In [3]:
with tf.Session() as sess:
    out = sess.run([node1, node2])
    print(out)

[6, 7]


In [4]:
node3 = tf.add(node1, node2)

def run_session(*args):
    with tf.Session() as sess:
        out = sess.run(args)
        return out

run_session(node3)

(13,)

In [8]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

res = a + b
mult = res * 3

with tf.Session() as sess:
    values = {a: [1, 3], b: [4, 5]}
    out = sess.run([mult], feed_dict=values)
    print(out)

[array([ 15.,  24.], dtype=float32)]


In [14]:
w = tf.Variable([0.3], dtype=tf.float32)
b = tf.Variable([-0.3], dtype=tf.float32)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

predicted_y = w*x + b
squared_loss = tf.square(predicted_y - y)
loss = tf.reduce_sum(squared_loss)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    values = {x: [1, 2, 3], y: [4, 8, 12]}
    out = sess.run(loss, feed_dict=values)
    print(out)

205.25


In [17]:
fixX = w.assign([4.0])
fixB = b.assign([0.0])

sess = tf.Session()
sess.run([fixX, fixB])
out = sess.run(loss, feed_dict=values)
print(out)

0.0


In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init)
for i in range(1000):
    sess.run(train, feed_dict=values)

print(sess.run([w, b]))

[array([ 3.99964809], dtype=float32), array([ 0.00079993], dtype=float32)]


In [20]:
sess.close()

In [51]:
## complete linear regression with tensorflow
import numpy as np
np.random.seed(0)

x_train = np.random.randn(10)
y_train = 2 * x_train + 10

x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
W = tf.Variable(tf.random_normal((1, )), dtype=tf.float32)
b = tf.Variable(tf.random_normal((1, )), dtype=tf.float32)

pred_y = W * x + b
squared_loss = tf.square(pred_y - y)
loss = tf.reduce_sum(squared_loss)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

epochs = 1000

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(epochs):
    sess.run(train, {x: x_train, y: y_train})

print(sess.run([W, b, loss], {x: x_train, y: y_train}))
sess.close()

[array([ 2.00000215], dtype=float32), array([ 9.99999619], dtype=float32), 8.2764018e-11]


In [52]:
tf.random_normal((10, ))

<tf.Tensor 'random_normal_60:0' shape=(10,) dtype=float32>

In [56]:
def model_fn(features, labels, mode):
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    
    y = W*features['x'] + b
    
    loss = tf.reduce_sum(tf.square(labels - y))
    
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train
    )

estimator = tf.estimator.Estimator(model_fn=model_fn)
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=train_input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Personal\\AppData\\Local\\Temp\\tmp3q3vrres', '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Personal\AppData\Local\Temp\tmp3q3vrres\model.ckpt.
INFO:tensorflow:step = 1, loss = 10.8243056552
INFO:tensorflow:global_step/sec: 720.429
INFO:tensorflow:step = 101, loss = 0.0300799469592 (0.140 sec)
INFO:tensorflow:global_step/sec: 815.747
INFO:tensorflow:step = 201, loss = 0.00270277654583 (0.122 sec)
INFO:tensorflow:global_step/sec: 861.457
INFO:tensorflow:step = 301, loss = 0.000242852856975 (0.117 sec)
INFO:tensorflow:global_step/sec: 780.689
INFO:tensorflow:step = 401, loss = 2.18210825575e-05 (0.128 sec)
INFO:tensorflow:global_step/sec: 

In [57]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
